# Train gradient boosting model using Bayesian optimization to find best hyperparameters

# 1. Imports

## 1.1 Packages

In [16]:
import sys

import pandas as pd


## 1.2 Options

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
sys.path.append('../src')

# from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
#     create_mlflow_experiment,
# )
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    add_lags_sma,
    get_split_train_val_cv,
    # train_model_bayesian_opti,
    train_model_mlflow,
)


In [19]:
lags_to_try = [1,]

In [20]:
feat_date = "duedate"

## 1.3 Datasets

In [21]:
df_train = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_train.sample(2)

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
6,170261730403893,17026,1,40,35,4,1,3,1,1,2024-10-31 19:44:53+00:00,75056,2024,10,31,3,0
2,140141730322556,14014,1,60,49,10,5,5,1,1,2024-10-30 21:09:16+00:00,75056,2024,10,30,2,0


In [22]:
df_train.rename(columns={"numbikesavailable": "target"}, inplace=True)

# 2. Prepare datasets

In [23]:
# Add lags defined
df_train = add_lags_sma(df_train, lags_to_try, feat_id='stationcode', feat_date=feat_date, feat_target="target", n_shift=5)

In [24]:
df_train.sort_values(by="duedate", inplace=True)
df_train.drop(columns="duedate", inplace=True)

In [25]:
list_df = get_split_train_val_cv(df_train, n_splits=2)

In [26]:
df_train, df_valid = list_df[0][0], list_df[0][1]

In [27]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [28]:
# experiment_id = create_mlflow_experiment(
#     experiment_folder_path="../data/06_models/mlruns",
#     experiment_name="velib_prediction"
# )
# experiment_id

experiment_id = '587291553688351204'

In [29]:
# search_params = {
#     "iterations": (100, 500),
#     "depth": (2, 7),
# }

params = {
    "iterations": 100,
    "depth": 4
}

In [30]:
# train_model_bayesian_opti(
#     run_name="Test_catboost",
#     experiment_id=experiment_id,
#     search_params=search_params,
#     df_train=df_train,
#     df_valid=df_valid,
#     feat_cat=feat_cat,
#     n_trials=10
# )

model, rmse_train, rmse_valid = train_model_mlflow(
    experiment_id=experiment_id,
    parent_run_id=None,
    df_train=df_train,
    df_valid=df_valid,
    feat_cat=feat_cat,
    **params
)

print("RMSE train:", rmse_train)
print("RMSE valid:", rmse_valid)

/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/26 22:05:49 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '                  idx stationcode  is_

RMSE train: 0.2992554793036776
RMSE valid: 0.7423017662117376
